In [ ]:
using PyCall
using Statistics
using LinearAlgebra
using Random
using Distributions
using TensorCrossInterpolation
import TensorCrossInterpolation: CachedFunction, optfirstpivot, crossinterpolate2
using JLD2
using Tensor_FixedSeedMC
import Tensor_FixedSeedMC as TCIMC

# Python パッケージのインポート
const np = pyimport("numpy")
@pyimport tf_quant_finance as tff
@pyimport tensorflow as tf

#-------------------------------------------------------------------------------
# モデル／グリッド生成ユーティリティ
#-------------------------------------------------------------------------------

"""
    chebyshev_lobatto_nodes(n::Int, a::Real, b::Real) -> (x_scaled, x_standard)

[-1,1] 上のチェビシェフ・ロバット節点を生成し、区間 [a,b] に線形写像したものを返す。
- `x_scaled`: [a,b] 上の節点
- `x_standard`: [-1,1] 上の節点
"""
function chebyshev_lobatto_nodes(n::Int, a::Real, b::Real)
    x_std = [cos(π * (n - 1 - i) / (n - 1)) for i in 0:n-1]
    x_scaled = 0.5 * (b - a) .* (x_std .+ 1) .+ a
    return x_scaled, x_std
end

#-------------------------------------------------------------------------------
# オプション価格計算
#-------------------------------------------------------------------------------

"""
    calculate_option_price(T, r, K, d, S0_indices, σ_grid, S0_grid, corr_mat, nPath, seed) -> (mean_pv, std_pv)

多変量幾何ブラウン運動モデルを使ってモンテカルロ価格を計算する。
- `T`: 満期
- `r`: 無リスク金利
- `K`: 行使価格
- `d`: 原資産数
- `S0_indices`: [σ1_idx, S0_1_idx, σ2_idx, …] のインデックス配列
- `σ_grid`: ボラティリティ候補の配列
- `S0_grid`: 初期株価候補の配列
- `corr_mat`: 相関行列 (d×d)
- `nPath`: サンプル数
- `seed`: 乱数シード
"""
function calculate_option_price(
    T::Float64,
    r::Float64,
    K::Float64,
    d::Int,
    S0_indices::Vector{Int},
    σ_grid::Vector{Float64},
    S0_grid::Vector{Float64},
    corr_mat::AbstractMatrix{Float64},
    nPath::Int,
    seed::Int
)
    # インデックスから各資産のパラメータを取り出し
    vols = [σ_grid[S0_indices[2*i-1]] for i in 1:d]
    S0s  = [S0_grid[S0_indices[2*i]]   for i in 1:d]

    # 乱数シード固定
    np.random.seed(seed)
    tf.random.set_seed(seed)

    # モデル生成 → サンプリング
    gbm = tff.models.MultivariateGeometricBrownianMotion(
        dim=d,
        means=fill(r, d),
        volatilities=vols,
        corr_matrix=corr_mat
    )

    paths = gbm.sample_paths(
        times=[T],
        initial_state=S0s,
        random_type=tff.math.random.RandomType.PSEUDO,
        seed=seed,
        num_samples=nPath
    ).numpy()  # shape: (num_samples, 1, d)

    # 最小価格バスケットオプションのペイオフ
    payoffs = max.(minimum(paths[:, 1, :], dims=2) .- K, 0.0)
    discount = exp(-r * T)
    mean_pv = discount * mean(payoffs)
    std_pv  = discount * std(payoffs)

    return mean_pv, std_pv
end

#-------------------------------------------------------------------------------
# 誤差検証スクリプト
#-------------------------------------------------------------------------------

"""
    monte_carlo_error_stats(
        reps, T, r, K, d, σ_grid, S0_grid, corr_mat, nPath, seed
    ) -> (mean_err, max_err)

複数回ランダムなグリッド選択で Monte Carlo の標準誤差を計算し、
平均誤差と最大誤差を返す。
"""
function monte_carlo_error_stats(
    reps::Int,
    T, r, K, d,
    σ_grid, S0_grid,
    corr_mat,
    nPath::Int,
    seed::Int
)
    abs_errs = Float64[]
    Random.seed!(seed)
    for _ in 1:reps
        idx = [rand(1:length(σ_grid)) for _ in 1:2*d]
        _, std_pv = calculate_option_price(T, r, K, d, idx, σ_grid, S0_grid, corr_mat, nPath, seed)
        push!(abs_errs, std_pv / sqrt(nPath))
    end
    return mean(abs_errs), maximum(abs_errs)
end

#-------------------------------------------------------------------------------
# TCI ワンショット補間
#-------------------------------------------------------------------------------

"""
    tci_oneshot(func, dims, firstpivot, tol; max_threads=4) -> (qtt, errors)

TCI のワンショット・クロス補間を実行する。
- `func(p)`: 評価可能な関数
- `dims`: 各次元ごとのグリッド数ベクトル
- `firstpivot`: 初期ピボット点
- `tol`: 許容誤差
"""
function tci_oneshot(
    func::Function,
    dims::Vector{Int},
    firstpivot::Vector{Int},
    tol::Float64;
    max_threads::Int=4
)
    BLAS.set_num_threads(max_threads)

    # 最適ピボット探索（簡易）
    for _ in 1:100
        p = optfirstpivot(func, dims, firstpivot)
        if abs(func(p)) > abs(func(firstpivot))
            firstpivot = p
        end
    end

    # クロス補間
    qtt, ranks, errors = crossinterpolate2(
        Float64, func, dims, [firstpivot];
        tolerance=tol,
        maxiter=1,
        verbosity=1,
        loginterval=1,
        pivotsearch=:rook,
        maxbonddim=1
    )

    return qtt, errors
end

#-------------------------------------------------------------------------------
# メイン実行部
#-------------------------------------------------------------------------------

function main()
    # パラメータ設定
    T        = 1.0
    r        = 0.01
    K        = 100.0
    d        = 5
    nPath    = 10^7  # 10^1
    seed     = 1234
    num_grid = 64

    # グリッド生成
    σ_grid, _  = chebyshev_lobatto_nodes(num_grid, 0.15, 0.25)
    S0_grid, _ = chebyshev_lobatto_nodes(num_grid, 90.0, 120.0)

    # 相関行列
    corr_mat = [
        1.0      0.303659  0.505532  0.719655  0.728832;
        0.303659 1.0      0.401077  0.515272  0.178203;
        0.505532 0.401077 1.0      0.132542  0.722767;
        0.719655 0.515272 0.132542 1.0      0.394723;
        0.728832 0.178203 0.722767 0.394723  1.0
    ]

    # ベースライン計算
    base_idx = fill(num_grid, 2*d)
    mean_pv, std_pv = calculate_option_price(
        T, r, K, d, base_idx, σ_grid, S0_grid, corr_mat, nPath, seed
    )
    println("Baseline Present Value: $mean_pv")
    println("Baseline StdDev: $std_pv")
    println("Baseline AbsErr: ", std_pv / sqrt(nPath))

    # 誤差統計
    mean_err, max_err = monte_carlo_error_stats(
        100, T, r, K, d, σ_grid, S0_grid, corr_mat, nPath, seed
    )
    println("Mean AbsErr over 100 runs: $mean_err")
    println("Max  AbsErr over 100 runs: $max_err")

    # TCI 補間
    abo = idx -> calculate_option_price(T, r, K, d, idx, σ_grid, S0_grid, corr_mat, nPath, seed)[1]
    dims       = fill(num_grid, 2*d)
    firstpivot = rand(1:num_grid, 2*d)
    tol_mc     = 1e-3

    phi = CachedFunction{Float64}(abo, dims)
    tt_tci, errors = tci_oneshot(phi, dims, firstpivot, tol_mc)
    println("TCI interpolation errors: ", errors)

    # 保存
    @save "tt_tci_$nPath.jld2" tt_tci
end

# スクリプト実行
main()


Baseline Present Value: 7.104700574394674


Baseline StdDev: 12.450613371403472
Baseline AbsErr: 0.012450613371403471
Mean AbsErr over 100 runs: 0.005000662917287731
Max  AbsErr over 100 runs: 0.010090917373199719


InterruptException: InterruptException: